In [ ]:
import openai
import pandas as pd
from tqdm import tqdm
import concurrent.futures
import time
import random
import difflib
import os

client = openai.OpenAI(
    api_key=""
)

# Đọc dữ liệu từ vnexpress_dataset.csv
df = pd.read_csv("./data/vnexpress_dataset.csv")

# Lọc tin trùng lặp từ bộ tin thật dựa trên Link
initial_count = len(df)
df = df.drop_duplicates(subset=["Link"], keep="first")
print(f"Đã loại bỏ {initial_count - len(df)} tin thật trùng lặp dựa trên Link. Còn lại {len(df)} tin thật.")

# (Tùy chọn) Lọc thêm trùng lặp dựa trên Content nếu cần
# def is_content_too_similar(content1, content2, threshold=0.95):
#     return difflib.SequenceMatcher(None, content1, content2).ratio() > threshold
#
# unique_contents = []
# unique_indices = []
# for idx, row in df.iterrows():
#     content = row["Content"]
#     if not any(is_content_too_similar(content, existing_content) for existing_content in unique_contents):
#         unique_contents.append(content)
#         unique_indices.append(idx)
# df = df.loc[unique_indices]
# print(f"Đã loại bỏ thêm {initial_count - len(unique_indices)} tin thật trùng lặp dựa trên Content. Còn lại {len(df)} tin thật.")

# Kiểm tra file tin giả đã tồn tại và lấy danh sách Link đã xử lý
processed_links = set()
fake_dataset_path = "./data/vnexpress_fake_dataset_enhance.csv"
if os.path.exists(fake_dataset_path):
    fake_df = pd.read_csv(fake_dataset_path)
    processed_links = set(fake_df["Link"].unique())
    print(f"Đã tìm thấy {len(processed_links)} Link đã được xử lý trong {fake_dataset_path}")

# Lọc các tin thật chưa được xử lý
df = df[~df["Link"].isin(processed_links)]
print(f"Còn lại {len(df)} tin thật chưa được xử lý để tạo tin giả")

# Hàm kiểm tra độ tương đồng giữa các nội dung
def is_too_similar(content1, content2, threshold=0.9):
    return difflib.SequenceMatcher(None, content1, content2).ratio() > threshold

# Hàm gọi API để tạo tin giả cho một batch tin thật
def generate_fake_news_batch(rows):
    try:
        # Chuẩn bị prompt cho batch
        prompt = ""
        for idx, row in enumerate(rows):
            content = row["Content"]
            prompt += f"""
            Tin thật {idx + 1}:
            [TIN THẬT]: {content}

            Hãy tạo 7 bản tin giả cho tin thật này. Mỗi bản tin giả phải:
            - Có nội dung sai sự thật nhưng hợp lý, đủ thuyết phục để có thể bị nhầm là thật.
            - Không sao chép nguyên văn tin thật, mà biến đổi câu chữ, thêm thông tin gây hiểu lầm hoặc bịa đặt.
            - Có văn phong giống báo lá cải (giật gân, hấp dẫn) hoặc bài đăng mạng xã hội (ngắn gọn, kích thích tương tác, sử dụng từ ngữ lan truyền).
            - Khác biệt rõ ràng so với tin thật và giữa các tin giả với nhau.
            - Không sử dụng các từ ngữ hoặc chi tiết vi phạm chính sách nội dung (bạo lực, phân biệt đối xử, v.v.).

            Định dạng đầu ra cho tin thật {idx + 1}:
            - Mỗi bản tin giả là một đoạn văn ngắn (2-4 câu), không đánh số, không sử dụng tiêu đề phụ.
            - Các đoạn cách nhau bằng một dòng trống.
            - Bắt đầu với dòng: "[Tin thật {idx + 1}]", theo sau là các bản tin giả.

            """
        prompt += "\nĐảm bảo các tin giả từ các tin thật khác nhau không trùng lặp nội dung."

        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.8,
        )

        fake_news = []
        current_tin_that_idx = None

        for choice in response.choices:
            # Tách nội dung theo các tin thật
            for line in choice.message.content.split("\n"):
                line = line.strip()
                if line.startswith("[Tin thật"):
                    try:
                        parts = line.split(" ")
                        if len(parts) > 2:
                            current_tin_that_idx = int(parts[2].strip("]")) - 1
                        else:
                            # If format is unexpected, skip this line
                            continue
                    except (ValueError, IndexError):
                        # If parsing fails, skip this line
                        continue
                    continue
                if line and current_tin_that_idx is not None:
                    # Check if current_tin_that_idx is valid for rows
                    if 0 <= current_tin_that_idx < len(rows):
                        row = rows[current_tin_that_idx]
                        content = row["Content"]
                        # Kiểm tra không trùng với tin thật
                        if not is_too_similar(line, content):
                            fake_news.append({
                                "Title": row["Title"],
                                "Link": row["Link"],
                                "Views": row["Views"],
                                "Comments": row["Comments"],
                                "Content": line,
                                "Label": 1
                            })
                    else:
                        # Skip if index is out of range
                        continue

        # Kiểm tra và loại bỏ tin giả trùng lặp
        unique_fake_news = []
        for news in fake_news:
            is_unique = True
            for existing_news in unique_fake_news:
                if is_too_similar(news["Content"], existing_news["Content"]):
                    is_unique = False
                    break
            if is_unique:
                unique_fake_news.append(news)

        return unique_fake_news
    except Exception as err:
        print(f"Error processing batch: {err}")
        return []

# Số lượng worker và batch size
MAX_WORKERS = 5
BATCH_SIZE = 3  # Số tin thật mỗi lần gửi API

fake_news_list = []

# Chỉ xử lý nếu còn tin thật chưa được tạo tin giả
if len(df) > 0:
    # Chia dữ liệu thành các batch
    batches = [df.iloc[i:i + BATCH_SIZE].to_dict('records') for i in range(0, len(df), BATCH_SIZE)]

    # Xử lý song song với ThreadPoolExecutor
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        # Tạo dictionary của các future với tham số đầu vào
        future_to_batch = {executor.submit(generate_fake_news_batch, batch): i for i, batch in enumerate(batches)}

        # Xử lý kết quả khi hoàn thành
        for future in tqdm(concurrent.futures.as_completed(future_to_batch), total=len(future_to_batch), desc="Generating fake news"):
            try:
                results = future.result()
                # Check if results is iterable before extending
                if isinstance(results, (list, tuple)) and results:
                    fake_news_list.extend(results)
                elif results and not isinstance(results, (list, tuple)):
                    print(f"Warning: Expected iterable but got {type(results)}")
                # Giảm tải API bằng cách đợi một chút giữa các yêu cầu
                time.sleep(random.uniform(0.3, 0.7))
            except Exception as e:
                print(f"Exception occurred: {e}")

    # Lưu tin giả vào file (nối thêm vào file hiện có)
    if fake_news_list:
        fake_df = pd.DataFrame(fake_news_list)
        if os.path.exists(fake_dataset_path):
            # Nối thêm vào file hiện có
            fake_df.to_csv(fake_dataset_path, mode='a', header=False, index=False, encoding="utf-8-sig")
        else:
            # Tạo file mới
            fake_df.to_csv(fake_dataset_path, index=False, encoding="utf-8-sig")
        print(f"✅ Đã tạo và lưu {len(fake_news_list)} tin giả mới vào {fake_dataset_path}")
else:
    print("⚠️ Không có tin thật mới nào để tạo tin giả")

# Xử lý mất cân bằng: Chọn ngẫu nhiên 2 tin giả từ mỗi tin thật
balanced_fake_news_list = []
fake_df = pd.read_csv(fake_dataset_path)  # Đọc lại file để lấy tất cả tin giả
for link in fake_df["Link"].unique():
    fake_subset = fake_df[fake_df["Link"] == link].to_dict('records')
    # Chọn ngẫu nhiên 2 tin giả từ mỗi Link nếu có ít nhất 1 tin giả
    if fake_subset:  # Check if fake_subset is not empty
        selected_fakes = random.sample(fake_subset, min(2, len(fake_subset)))
        balanced_fake_news_list.extend(selected_fakes)

# Tạo tập dữ liệu tin thật (sử dụng bộ tin thật đã lọc trùng lặp)
real_news_list = [
    {
        "Title": row["Title"],
        "Link": row["Link"],
        "Views": row["Views"],
        "Comments": row["Comments"],
        "Content": row["Content"],
        "Label": 0
    } for _, row in df.iterrows()
]

# Kết hợp tin thật và tin giả
combined_news_list = real_news_list + balanced_fake_news_list
combined_df = pd.DataFrame(combined_news_list)

# Lưu vào file kết hợp
combined_df.to_csv("./data/vnexpress_combined_dataset.csv", index=False, encoding="utf-8-sig")

print(f"✅ Đã tạo và lưu {len(combined_news_list)} bản tin (thật + giả) thành công vào vnexpress_combined_dataset.csv")

Đã loại bỏ 62 tin thật trùng lặp dựa trên Link. Còn lại 6473 tin thật.
Đã tìm thấy 1610 Link đã được xử lý trong ./data/vnexpress_fake_dataset_enhance.csv
Còn lại 4863 tin thật chưa được xử lý để tạo tin giả


Generating fake news:   6%|▋         | 104/1621 [05:20<1:38:56,  3.91s/it]

Error processing batch: list index out of range


Generating fake news:  14%|█▍        | 224/1621 [11:23<1:26:31,  3.72s/it]

Error processing batch: list index out of range


...

Generating fake news:  85%|████████▌ | 1383/1621 [1:03:18<18:16,  4.61s/it]

Error processing batch: list index out of range


Generating fake news:  90%|█████████ | 1467/1621 [1:07:21<07:38,  2.98s/it]

Error processing batch: list index out of range


Generating fake news:  91%|█████████ | 1475/1621 [1:07:45<06:44,  2.77s/it]